# Model Selection

### Import libreries

In [1]:
import os
import typing

from sklearn.gaussian_process.kernels import *
from sklearn.kernel_approximation import Nystroem
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import pandas as pd

### Read data

In [2]:
X = pd.read_csv("./train_x.csv").to_numpy()
y = pd.read_csv("./train_y.csv").to_numpy()

### Sample data

In [20]:
data = pd.read_csv("./train_x.csv")
data['y'] = pd.read_csv("./train_y.csv").to_numpy()
data = data.sample(frac = 0.1, replace=True)

In [48]:
X = data.iloc[:,0:2].to_numpy()
y = data['y'].to_numpy()

### Nystroem Approximation

In [3]:
feature_map_nystroem = Nystroem(random_state=1,
                                n_components=round(y.size * 0.1))

In [5]:
feature_map_nystroem.fit(X,y)
transformed_X = feature_map_nystroem.transform(X)

In [9]:
feature_map_nystroem.get_params(deep=True)

array([[0.02444543, 0.01090247, 0.0395247 , ..., 0.01165562, 0.01168586,
        0.01103407],
       [0.03067112, 0.01891697, 0.02447058, ..., 0.024786  , 0.02659303,
        0.02630992],
       [0.02388203, 0.00665553, 0.04094812, ..., 0.00839871, 0.01004779,
        0.01089997],
       ...,
       [0.02770942, 0.02296312, 0.01481611, ..., 0.03517939, 0.03995241,
        0.04122893],
       [0.02682245, 0.01698875, 0.01350823, ..., 0.03164337, 0.04153956,
        0.04761982],
       [0.0220179 , 0.04282287, 0.01439282, ..., 0.03420879, 0.0261344 ,
        0.02086419]])

### Toy example for RBF

In [52]:
kernel = RBF()
gpc = GaussianProcessRegressor(kernel=kernel, random_state=0, n_restarts_optimizer=50).fit(X, y)
gpc.log_marginal_likelihood()

-414408.8566421233

In [53]:
gpc.kernel_

RBF(length_scale=0.0119)

In [13]:
gpc.log_marginal_likelihood()

-10974414.29929122

In [ ]:
gpc.log_marginal_likelihood()

In [14]:
gpc.get_params(deep=True)

{'alpha': 1e-10,
 'copy_X_train': True,
 'kernel__length_scale': 1.0,
 'kernel__length_scale_bounds': (1e-05, 100000.0),
 'kernel': RBF(length_scale=1),
 'n_restarts_optimizer': 1,
 'normalize_y': False,
 'optimizer': 'fmin_l_bfgs_b',
 'random_state': 0}